Visual models are best for people, numerical models are for machines.  Bokeh's visual interactions and static embedding make it the best candidate for data science.  Interacting with information allows the viewer to draw their own unique insights; static visualizations will only tell the creator's story.

This notebook designs a Object Oriented/Functional API for interactive grid plots in Bokeh; this extends a [previous post](https://foreverwhatever.github.io/2017/06/29/GridPlot-In-Bokeh.html).

In [1]:
    import pandas as pd
    from bokeh import plotting, models, layouts, resources
    from operator import gt, lt, eq
    from functools import wraps
    from coffeetools import coffee
    plotting.output_notebook()

    import collections

In [4]:
    df = pd.util.testing.makeDataFrame().reset_index(drop=True)

In [28]:
    class DeepChainMap(collections.ChainMap):
        def __setitem__(self, key, value):
            for mapping in self.maps:
                if key in mapping:
                    mapping[key] = value
                    return
            self.maps[0][key] = value

        def __delitem__(self, key):
            for mapping in self.maps:
                if key in mapping:
                    del mapping[key]
                    return
            raise KeyError(key)

In [44]:
    def create_sources(df, agg=pd.Series.describe):
        return DeepChainMap(
            {None: df}
        ).new_child(
            {None: df.pipe(plotting.ColumnDataSource)}
        ).new_child(
            {_: df[_].pipe(agg) for _ in df.columns})

In [45]:
    create_sources(df)[None]

ColumnDataSource(id='903a0161-e14e-49c7-b21e-ab8d2fb8cfe8', ...)

In [46]:
    class Plots(object):
        def __init__(self, df, features=None):
            self.sources = create_sources(df)
            self.features = features or list(df.columns)
            self.figures = collections.defaultdict(plotting.Figure)
            
        

SyntaxError: unexpected EOF while parsing (<ipython-input-46-98293e82d216>, line 8)

* Every figure is a collection of figures.  
* A single figure is one row and column.
* Every Glyph requires a data source to exist first.
  * Use __getitem__ method to access data source.  
* Apply glyphs to a data source.  The data source will append glyph to figures because on some iterables.
    

In [ ]:
    def bintrue(a, b):
        return True

In [ ]:
    from operator import eq, lt, gt

In [ ]:
    def replaceRowCol(obj, **kwargs):
        if isinstance(obj, str):
            obj = obj.format(**kwargs)
        if isinstance(obj, dict):
            obj = {k: replaceRowCol(v, **kwargs) for k, v in obj.items()} 
        return obj

In [ ]:
    class Missing:
        def __missing__(self, key):
            self[key] = self.default()
            return self[key]

    class Iter:
        def __init__(self, sources=None, rows=None, columns=None, figures=None, glyph=None):
            if isinstance(sources, plotting.ColumnDataSource):
                sources = {None: sources}
            self.sources, self.figures = sources or Sources(), figures or Figures()
            self.rows, self.columns, self.glyph = rows, columns or rows, glyph
            rows and [self.figures[r][c] for r, c in self]
            
        def __iter__(self):
            for i, row in enumerate(self.rows or self.figures):
                for j, col in enumerate(self.columns or self.figures[row]):
                    
                    if self.cmp(i, j):
                        yield row, col
            
        def __call__(self, **kwargs):
            for source in self.sources.values():
                for row, col in self:
                    self.figures[row][col].add_glyph(
                        source, self.glyph(**replaceRowCol(kwargs, row=row, col=col)))
            return self
                        
        def __getattr__(self, attr):
            self.glyph = getattr(models, attr)
            return self
        
    class Diagonal(Iter):
        cmp = staticmethod(eq)
        
    class Upper(Iter):
        cmp = staticmethod(lt)

    class Lower(Iter):
        cmp = staticmethod(gt)

    class All(Iter):
        cmp = staticmethod(bintrue)

In [ ]:
    from collections import ChainMap

In [ ]:
    class Explore(object):
        class __init__

* __figures__

    The keys are rows and columns.
   
   
* There are multiple datasources to support categorical data and the diagonals.


In [ ]:
    class Row(collections.OrderedDict, Missing):
        default = staticmethod(lambda **kwargs: plotting.Figure(
            **kwargs, width=200, height=200
        ))

    class Figures(collections.OrderedDict, Missing):
        default = staticmethod(Row)

    class Sources(collections.OrderedDict, Missing):
        default = staticmethod(models.ColumnDataSource)
        
    class Grid(All):
        def __getitem__(self, key=None, cls=All):
            return cls(self.sources[key], self.rows, self.columns, self.figures)
        
        @property
        def diagonal(self):
            return self.__getitem__(cls=Diagonal)

        @property
        def upper(self):
            return self.__getitem__(cls=Upper)

        @property
        def lower(self):
            return self.__getitem__(cls=Lower)

        @property
        def all(self):
            return self.__getitem__(cls=All)

        @property
        def _figures(self):
            [self.figures[r][c] for r, c in self]
            return 
        
        def show(self):
            return layouts.gridplot(self._figures, ncols=len(self.figures))

In [ ]:
    df = pd.util.testing.makeDataFrame().reset_index(drop=True)

In [ ]:
    g = Grid(df.pipe(plotting.ColumnDataSource), ['A', 'B', 'C'])

    g.upper.Circle(x='{row}', y='{col}')
    g.lower.X(x='{row}', y='{col}')

    plotting.show(g.show())